In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk

import pandas as pd 
import numpy as np

# CountVectorizer will help calculate word counts
from sklearn.feature_extraction.text import CountVectorizer

# Import the string dictionary that we'll use to remove punctuation
import string

In [2]:
data = pd.read_csv('reviews.csv')


In [3]:
data = data[:2000]

In [8]:
data.head(2)

Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW  delmartian                     1   
1   2  B00813GRG4  A1D87F6ZCVE5NK      dll pa                     0   

   HelpfulnessDenominator  Score        Time                Summary  \
0                       1      5  1303862400  Good Quality Dog Food   
1                       0      1  1346976000      Not as Advertised   

                                                Text  
0  I have bought several of the Vitality canned d...  
1  Product arrived labeled as Jumbo Salted Peanut...

## Generating review sentiments

In [9]:
data.rename(columns={"Score": "sentiment"}, inplace=True)
data.loc[data["sentiment"] < 3, "sentiment"] = 0
data.loc[data["sentiment"] > 2, "sentiment"] = 1


In [10]:
stop = set(stopwords.words('english'))
#print(stop)
punc = set(string.punctuation)
#keywords = data['brand'].apply(lambda x: str(x).lower()).unique().tolist()
#keywords.append('phone')
lemma = WordNetLemmatizer()
def clean_text(text):
    # Convert the text into lowercase
    text = text.lower()
    # Split into list
    wordList = text.split()
    # Remove punctuation
    wordList = ["".join(x for x in word if (x=="'")|(x not in punc)) for word in wordList]
    # Remove stopwords
    wordList = [word for word in wordList if word not in stop]
    # Remove other keywords
    #wordList = [word for word in wordList if word not in keywords]
    # Lemmatisation
    wordList = [lemma.lemmatize(word) for word in wordList]
    return " ".join(wordList)
clean_text("I love reading books.")

'love reading book'

In [11]:
#data["body"] = data["body"].astype("str")
data["clean_text"] = data["Text"].apply(clean_text)
cleaned_data = pd.DataFrame(data["Text"])
cleaned_data = cleaned_data.join(data["clean_text"])
cleaned_data = cleaned_data.join(data["Summary"])
cleaned_data = cleaned_data.join(data["sentiment"])



## New table created based on requirements

In [42]:
cleaned_data.head(5)

Text  \
0  I have bought several of the Vitality canned d...   
1  Product arrived labeled as Jumbo Salted Peanut...   
2  This is a confection that has been around a fe...   
3  If you are looking for the secret ingredient i...   
4  Great taffy at a great price.  There was a wid...   

                                          clean_text                Summary  \
0  bought several vitality canned dog food produc...  Good Quality Dog Food   
1  product arrived labeled jumbo salted peanutsth...      Not as Advertised   
2  confection around century light pillowy citrus...  "Delight" says it all   
3  looking secret ingredient robitussin believe f...         Cough Medicine   
4  great taffy great price wide assortment yummy ...            Great taffy   

   sentiment  
0          1  
1          0  
2          1  
3          0  
4          1

Algorithm for weight calculation:

For each class  j∈{positive,neutral,negative} 

a. Find all the words  i  in the reviews belonging to class  j .

b. Calculate  ni,j=  the number of reviewss in class  j  containing word  i .

c. Let  dj  be the number of reviews in class  j . Calculate  pi,j=ni,jdj , the proportion of reviews in class  j  that conain word  i .

d. Let  wi,j=pi,j−∑k≠jpi,k  be the weights assigned to each word within each class.

In [13]:
X_train,X_test = train_test_split(cleaned_data,train_size = 0.8)

In [14]:
pos_train = X_train[X_train['sentiment'] == 1]

neg_train = X_train[X_train['sentiment'] == 0]

In [15]:
cv = CountVectorizer(max_df=0.95, min_df=2, max_features=10000,stop_words='english')

In [16]:

X_train_cv = cv.fit_transform(X_train['clean_text'])

X_pos = cv.transform(pos_train['clean_text'])

X_neg = cv.transform(neg_train['clean_text'])

pos_count_df = pd.DataFrame(X_pos.toarray(), columns=cv.get_feature_names())

neg_count_df = pd.DataFrame(X_neg.toarray(), columns=cv.get_feature_names())

In [17]:

pos_words = {}

neg_words = {}

for k in cv.get_feature_names():
    pos = pos_count_df[k].sum()

    neg = neg_count_df[k].sum()
    
    pos_words[k] = pos/pos_train.shape[0]

    neg_words[k] = neg/neg_train.shape[0]

In [18]:
neg_words_adj = {}
pos_words_adj = {}

for key, value in neg_words.items():
    neg_words_adj[key] = neg_words[key] -  pos_words[key]
    
for key, value in pos_words.items():
    pos_words_adj[key] = pos_words[key] - neg_words[key]
    

Algorithm for finding selected text:
For every review:

a. Let  j  be the sentiment of the reciew.

b. If  j==  neutral return entire text.

c. Otherwise, for each subset of words in the review, calculate  ∑iwi,j , where  i  is the set of words in the review

d. Return the subset of words with the largest sum, given that it exceeds some tolerance.



In [19]:
def calculate_selected_text(df_row, tol = 0):
    
    review = df_row['clean_text']
    sentiment = df_row['sentiment']
    
    
    if(sentiment == 1):
        dict_to_use = pos_words_adj # Calculate word weights using the pos_words dictionary
    elif(sentiment == 0):
        dict_to_use = neg_words_adj # Calculate word weights using the neg_words dictionary
        
    words = review.split()
    words_len = len(words)
    subsets = [words[i:j+1] for i in range(words_len) for j in range(i,words_len)]
    
    score = 0
    selection_str = '' # This will be our choice
    lst = sorted(subsets, key = len) # Sort candidates by length
    
    for i in range(len(subsets)):
        
        new_sum = 0 # Sum for the current substring
        
        # Calculate the sum of weights for each word in the substring
        for p in range(len(lst[i])):
            if(lst[i][p].translate(str.maketrans('','',string.punctuation)) in dict_to_use.keys()):
                new_sum += dict_to_use[lst[i][p].translate(str.maketrans('','',string.punctuation))]
            
        # If the sum is greater than the score, update our current selection
        if(new_sum > score + tol):
            score = new_sum
            selection_str = lst[i]
            #tol = tol*5 # Increase the tolerance a bit each time we choose a selection

    # If we didn't find good substrings, return the whole text
    if(len(selection_str) == 0):
        selection_str = words
        
    return ' '.join(selection_str)

In [20]:
pd.options.mode.chained_assignment = None

In [22]:
tol = 0.001

X_test['predicted_selection'] = ''

for index, row in X_test.iterrows():
    
    selected_text = calculate_selected_text(row, tol)
    
    X_test.loc[X_test['Summary'] == row['Summary'], ['predicted_selection']] = selected_text

In [48]:
sentiment_extracted = X_test.iloc[70,:]

In [50]:
print("Original:","\t"+sentiment_extracted.Text)
print()
print("Extracted:","\t"+sentiment_extracted.predicted_selection)

Original: 	I bought these for my husband and he said they are the best energy shots out there. He takes one in the mornings and works hard all day. Good stuff!

Extracted: 	best energy shot take one morning work hard day good


In [51]:
sentiment_extracted = X_test.iloc[100,:]

In [53]:
print("Original:","\t"+sentiment_extracted.Text)
print()
print("Extracted:","\t"+sentiment_extracted.predicted_selection)

Original: 	Despite coming in an extremely large box, I found this to be great value. All the bags were preserved with a reasonable expiration date that is months away. I'd definitely recommend.

Extracted: 	great value


In [40]:
sentiment_extracted.clean_text

"despite coming extremely large box found great value bag preserved reasonable expiration date month away i'd definitely recommend"

In [41]:
sentiment_extracted.predicted_selection

'great value'

In [36]:
a = X_test.iloc[200,:]

In [37]:
a.clean_text

'phone new shipped fast exaclty though would really love phone working great'

In [38]:
a.predicted_selection

'fast exaclty though would really love phone working great'